# 1. Environment Setup

In [ ]:
%pip install -q langchain langchain-nvidia-ai-endpoints gradio langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
import os
os.environ["NVIDIA_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

- prints intermediate states in a more readable format using Python's pprint module (which is useful for pretty-printing complex data structures like dictionaries and lists).

In [ ]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [ ]:
%%time

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from typing import List, Union

# Zero shot classification prompt

## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
instruct_chat = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser()

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or go from under the bridge?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of height, so probably isn't for me"))

--------------------------------------------------------------------------------
car
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
air
CPU times: user 38.9 ms, sys: 1.94 ms, total: 40.9 ms
Wall time: 659 ms


In [ ]:
%%time

gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    {'topic' : zsc_chain}
    | PPrint()
    | gen_chain
)

chain.invoke({"input" : input_msg, "options" : options})

State: 
{'topic': ' boat'}

CPU times: user 29.6 ms, sys: 1.04 ms, total: 30.7 ms
Wall time: 735 ms


" As the sun began to set, the children's eyes gleamed with excitement as they rowed their makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and magic."

In [ ]:
%%time

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | PPrint()
    | RunnableAssign({'generation' : gen_chain})
    | PPrint()
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})

pprint("Final Output: ", output)

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown']
}

State: 
{'input': "I get seasick, so I think I'll pass on the trip", 'topic': ' boat'}

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': ' boat',
    'generation': " As the sun began to set, the children's eyes gleamed with excitement as they rowed their 
makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and 
magic."
}

Final Output: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': ' boat',
    'generation': " As the sun began to set, the children's eyes gleamed with excitement as they rowed their 
makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and 
magic.",
    'combination': " I enjoy watching children's excitement as they play with paper boats in bathwater, but I get 
seasick myself so won't join them."
}

CPU times: user 54.6 ms, sys: 9.4 ms, total: 64 ms
Wall time: 1.21 s


# Running State Chain

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pprint import pprint
from typing import Dict, Union, Optional, List

# First, determine which version of Pydantic you're using
try:
    # For Pydantic v2
    from pydantic import BaseModel, Field, create_model

    class KnowledgeBase(BaseModel):
        topic: str = Field(default='general', description="Current conversation topic")
        user_preferences: Dict[str, Union[str, int, List[str]]] = Field(default_factory=dict, description="User preferences and choices")  # Updated line
        session_notes: str = Field(default="", description="Notes on the ongoing session")
        unresolved_queries: List[str] = Field(default_factory=list, description="Unresolved user queries")
        action_items: List[str] = Field(default_factory=list, description="Actionable items identified during the conversation")

        @classmethod
        def schema(cls):
            return cls.model_json_schema()

except (ImportError, AttributeError):
    # For Pydantic v1
    from langchain.pydantic_v1 import BaseModel, Field

    class KnowledgeBase(BaseModel):
        topic: str = Field(default='general', description="Current conversation topic")
        user_preferences: Dict[str, Union[str, int, List[str]]] = Field(default_factory=dict, description="User preferences and choices")  # Updated line
        session_notes: str = Field(default="", description="Notes on the ongoing session")
        unresolved_queries: List[str] = Field(default_factory=list, description="Unresolved user queries")
        action_items: List[str] = Field(default_factory=list, description="Actionable items identified during the conversation")

        @classmethod
        def model_json_schema(cls):
            return cls.schema()

# Initialize the PydanticOutputParser with the KnowledgeBase model
output_parser = PydanticOutputParser(pydantic_object=KnowledgeBase)

# Get the format instructions from the output parser
try:
    instruct_string = output_parser.get_format_instructions()
    pprint(instruct_string)

except Exception as e:
    print(f"Error getting format instructions: {e}")

    # Alternative approach for older LangChain versions
    print("\nTrying alternative approach...")
    from langchain.prompts import PromptTemplate

    template = """
    Generate a response in the following format:

    topic: The current conversation topic
    user_preferences: A dictionary of user preferences
    session_notes: Notes about the ongoing conversation
    unresolved_queries: A list of unresolved queries
    action_items: A list of action items
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=[]
    )

    print(prompt.format())

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"topic": {"default": "general", "description": "Current '
 'conversation topic", "title": "Topic", "type": "string"}, '
 '"user_preferences": {"additionalProperties": {"anyOf": [{"type": "string"}, '
 '{"type": "integer"}, {"items": {"type": "string"}, "type": "array"}]}, '
 '"description": "User preferences and choices", "title": "User Preferences", '
 '"type": "object"}, "session_notes": {"default": "", "description": "Notes on '
 'the ongoing session", "title": "Sessio

In [ ]:

## Definition of RExtract
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

## Practical Use of RExtract

parser_prompt = ChatPromptTemplate.from_template(
    "Update the knowledge base: {format_instructions}. Only use information from the input."
    "\n\nNEW MESSAGE: {input}"
)

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)

knowledge = extractor.invoke({'input' : "I love flowers so much! The orchids are amazing! Can you buy me some?"})
pprint(knowledge)

KnowledgeBase(topic='general', user_preferences={}, session_notes='User expressed interest in buying orchids', unresolved_queries=['Can you buy the user some orchids?'], action_items=[])


In [ ]:
class KnowledgeBase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, unknown if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, unknown if unknown")
    location: str = Field('unknown', description="Where the user is located")
    summary: str = Field('unknown', description="Running summary of conversation. Update this with new input")
    response: str = Field('unknown', description="An ideal response to the user based on their new message")


In [ ]:
parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded ('input'). Please update the knowledge base."
    " Record your response in the 'response' tag to continue the conversation."
    " Do not hallucinate any details, and make sure the knowledge base is not redundant."
    " Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)


In [ ]:
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

## Initialize the knowledge base and see what you get
state = {'know_base' : KnowledgeBase()}
state['input'] = "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere in the United States."
state = info_update.invoke(state)
pprint(state)

{'input': "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere "
          'in the United States.',
 'know_base': KnowledgeBase(firstname='Carmen', lastname='Sandiego', location='unknown', summary='The user introduced themselves as Carmen Sandiego and asked for a guess on their location within the United States, providing a hint.', response="Welcome, Carmen Sandiego! I'm excited to try and guess your location. Since you mentioned it's somewhere in the United States, I'll start there. Is it by any chance in New York City?")}


In [ ]:
state['input'] = "I'm in a place considered the birthplace of jesus christ."
state = info_update.invoke(state)
pprint(state)

{'input': "I'm in a place considered the birthplace of jesus christ.",
 'know_base': KnowledgeBase(firstname='Carmen', lastname='Sandiego', location='unknown', summary='The user introduced themselves as Carmen Sandiego and asked for a guess on their location within the United States, providing a hint. The user revealed that they are in a place considered the birthplace of Jesus Christ.', response="Ah, I see! You're not in the United States then. If you're in the place considered the birthplace of Jesus Christ, that means you must be in Bethlehem, located in the West Bank.")}


In [ ]:

state['input'] = "Yeah, I'm in New Orleans... How did you know?"
state = info_update.invoke(state)
pprint(state)

{'input': "Yeah, I'm in New Orleans... How did you know?",
 'know_base': KnowledgeBase(firstname='Carmen', lastname='Sandiego', location='New Orleans', summary='The user initially introduced themselves as Carmen Sandiego and asked for a guess on their location within the United States, providing a hint. They later revealed that they are in a place considered the birthplace of Jesus Christ, which turned out to be a misdirection. The user admitted they were actually in New Orleans.', response="Oh, I'm sorry for the confusion earlier! I thought you were in Bethlehem based on your initial hint. But now that you've revealed your true location, I can see you're in New Orleans! It's a fantastic city with its rich history, unique culture, and delicious food. How can I assist you further in your visit?")}
